In [42]:
import muspy
from harmony_tokenizers import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, \
    GCTRootPCTokenizer, GCTSymbolTokenizer, GCTRootTypeTokenizer
from tqdm import tqdm
import os
import numpy as np
import zlib

In [43]:
gjt_path = '/media/maximos/9C33-6BBD/data/gjt_melodies/Library_melodies/'
gjt_list = os.listdir(gjt_path)
print(len(gjt_list))

650


In [44]:
gjt_pieces = []
for i in tqdm(range(len(gjt_list))):
    g = muspy.read_musicxml(gjt_path + gjt_list[i])
    gjt_pieces.append(g)

  1%|          | 6/650 [00:00<00:12, 50.34it/s]Exception ignored in: <function ZipFile.__del__ at 0x7073d23bc680>
Traceback (most recent call last):
  File "/home/maximos/miniconda/envs/midi/lib/python3.11/zipfile.py", line 1874, in __del__
    self.close()
  File "/home/maximos/miniconda/envs/midi/lib/python3.11/zipfile.py", line 1892, in close
    self._write_end_record()
  File "/home/maximos/miniconda/envs/midi/lib/python3.11/zipfile.py", line 1986, in _write_end_record
    self.fp.write(endrec)
TypeError: string argument expected, got 'bytes'
100%|██████████| 650/650 [00:14<00:00, 46.38it/s]


In [45]:
print(len(gjt_pieces))

650


In [58]:
# prepare stats
stats = {}

def compute_compression_rate(array: np.ndarray, compression_method=zlib.compress) -> float:
    """
    Compute the compression rate of a NumPy array.

    Parameters:
        array (np.ndarray): The NumPy array to compress.
        compression_method (callable): The compression method to use. 
                                       Default is `zlib.compress`.

    Returns:
        float: The compression rate (compressed size / original size).
    """
    # Convert the array to bytes
    array_bytes = array.tobytes()
    
    # Compress the byte representation
    compressed_bytes = compression_method(array_bytes)
    
    # Compute sizes
    original_size = len(array_bytes)
    compressed_size = len(compressed_bytes)
    
    # Calculate compression rate
    compression_rate = compressed_size / original_size

    return compression_rate

def initialize_stats(key, tokenizer):
    stats[key] = {
        'vocab_size': len(tokenizer.vocab),
        'seq_lens': [],
        'compression_rates': []
    }
# end initialize_stats

def update_stats(key, toks):
    for t in toks['ids']:
        stats[key]['seq_lens'].append( len(t) )
        stats[key]['compression_rates'].append( compute_compression_rate(np.array(t)) )
    stats[key]['mean_len'] = np.mean(stats[key]['seq_lens'])
    stats[key]['std_len'] = np.std(stats[key]['seq_lens'])
    stats[key]['mean_compression'] = np.mean(stats[key]['compression_rates'])
    stats[key]['std_compression'] = np.std(stats[key]['compression_rates'])
# end update_stats

def print_stats(key):
    print('mean: ', stats[key]['mean_len'])
    print('std: ', stats[key]['std_len'])
    print('mean cr: ', stats[key]['mean_compression'])
    print('std cr: ', stats[key]['std_compression'])

In [59]:
print('ChordSymbolTokenizer')
chordSymbolTokenizer = ChordSymbolTokenizer()
print('len(chordSymbolTokenizer.vocab): ', len(chordSymbolTokenizer.vocab))
# stats['ChordSymbolTokenizer'] = {
#     'vocab_size': len(chordSymbolTokenizer.vocab),
#     'seq_lens': [],
#     'compression_rates': []
# }
initialize_stats('ChordSymbolTokenizer', chordSymbolTokenizer)
toks_cs = chordSymbolTokenizer(gjt_pieces)
print('example sentence length: ', len(toks_cs['tokens'][0]))
print(toks_cs['tokens'][0])
print(toks_cs['ids'][0])
update_stats('ChordSymbolTokenizer', toks_cs)
# for t in toks_cs['tokens']:
#     stats['ChordSymbolTokenizer']['seq_lens'].append( len(t) )
# stats['ChordSymbolTokenizer']['mean_len'] = np.mean(stats['ChordSymbolTokenizer']['seq_lens'])
# stats['ChordSymbolTokenizer']['std_len'] = np.std(stats['ChordSymbolTokenizer']['seq_lens'])
print_stats('ChordSymbolTokenizer')
# print('mean: ', stats['ChordSymbolTokenizer']['mean_len'])
# print('std: ', stats['ChordSymbolTokenizer']['std_len'])
# print('mean cr: ', stats['ChordSymbolTokenizer']['mean_compression'])
# print('std cr: ', stats['ChordSymbolTokenizer']['std_compression'])

ChordSymbolTokenizer
len(chordSymbolTokenizer.vocab):  370
example sentence length:  110
['bar', 'position_0x0', 'G:min7', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'G:min7', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'E:min7', 'bar', 'position_0x0', 'A:7', 'bar', 'position_0x0', 'D:min7', 'bar', 'position_0x0', 'G:7', 'bar', 'position_0x0', 'C:maj7', 'bar', 'position_0x0', 'D:min7', 'position_2x0', 'G:7', 'bar', 'position_0x0', 'G:min7', 'position_0x0', 'C:7', 'position_0x0', 'F:maj', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'G:min7', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'G:min7', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'A:maj7', 'position_2x0', 'A#:maj7', 'bar', '

In [8]:
print('RootTypeTokenizer')
rootTypeTokenizer = RootTypeTokenizer()
print('len(rootTypeTokenizer.vocab): ', len(rootTypeTokenizer.vocab))
stats['RootTypeTokenizer'] = {
    'vocab_size': len(rootTypeTokenizer.vocab),
    'seq_lens': []
}
toks_rt = rootTypeTokenizer(gjt_pieces)
print('example sentence length: ', len(toks_rt['tokens'][0]))
print(toks_rt['tokens'][0])
print(toks_rt['ids'][0])
for t in toks_rt['tokens']:
    stats['RootTypeTokenizer']['seq_lens'].append( len(t) )
stats['RootTypeTokenizer']['mean_len'] = np.mean(stats['RootTypeTokenizer']['seq_lens'])
stats['RootTypeTokenizer']['std_len'] = np.std(stats['RootTypeTokenizer']['seq_lens'])
print('mean: ', stats['RootTypeTokenizer']['mean_len'])
print('std: ', stats['RootTypeTokenizer']['std_len'])

RootTypeTokenizer
len(rootTypeTokenizer.vocab):  63
example sentence length:  149
['bar', 'position_0x0', 'G', 'min7', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'G', 'min7', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'E', 'min7', 'bar', 'position_0x0', 'A', '7', 'bar', 'position_0x0', 'D', 'min7', 'bar', 'position_0x0', 'G', '7', 'bar', 'position_0x0', 'C', 'maj7', 'bar', 'position_0x0', 'D', 'min7', 'position_2x0', 'G', '7', 'bar', 'position_0x0', 'G', 'min7', 'position_0x0', 'C', '7', 'position_0x0', 'F', 'maj', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'G', 'min7', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'G', 'min7', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'F', 'm

In [10]:
print('PitchClassTokenizer')
pitchClassTokenizer = PitchClassTokenizer()
print('len(pitchClassTokenizer.vocab): ', len(pitchClassTokenizer.vocab))
stats['PitchClassTokenizer'] = {
    'vocab_size': len(pitchClassTokenizer.vocab),
    'seq_lens': []
}
toks_pc = pitchClassTokenizer(gjt_pieces)
print('example sentence length: ', len(toks_pc['tokens'][0]))
print(toks_pc['tokens'][0])
print(toks_pc['ids'][0])
for t in toks_pc['tokens']:
    stats['PitchClassTokenizer']['seq_lens'].append( len(t) )
stats['PitchClassTokenizer']['mean_len'] = np.mean(stats['PitchClassTokenizer']['seq_lens'])
stats['PitchClassTokenizer']['std_len'] = np.std(stats['PitchClassTokenizer']['seq_lens'])
print('mean: ', stats['PitchClassTokenizer']['mean_len'])
print('std: ', stats['PitchClassTokenizer']['std_len'])

PitchClassTokenizer
len(pitchClassTokenizer.vocab):  34
example sentence length:  226
['bar', 'position_0x0', 'chord_pc_7', 'chord_pc_10', 'chord_pc_2', 'chord_pc_5', 'bar', 'position_0x0', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7', 'chord_pc_10', 'bar', 'position_0x0', 'chord_pc_5', 'chord_pc_9', 'chord_pc_0', 'chord_pc_4', 'bar', 'position_0x0', 'chord_pc_5', 'chord_pc_9', 'chord_pc_0', 'chord_pc_4', 'bar', 'position_0x0', 'chord_pc_7', 'chord_pc_10', 'chord_pc_2', 'chord_pc_5', 'bar', 'position_0x0', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7', 'chord_pc_10', 'bar', 'position_0x0', 'chord_pc_5', 'chord_pc_9', 'chord_pc_0', 'chord_pc_4', 'bar', 'position_0x0', 'chord_pc_5', 'chord_pc_9', 'chord_pc_0', 'chord_pc_4', 'bar', 'position_0x0', 'chord_pc_4', 'chord_pc_7', 'chord_pc_11', 'chord_pc_2', 'bar', 'position_0x0', 'chord_pc_9', 'chord_pc_1', 'chord_pc_4', 'chord_pc_7', 'bar', 'position_0x0', 'chord_pc_2', 'chord_pc_5', 'chord_pc_9', 'chord_pc_0', 'bar', 'position_0x0', 'chord_pc_7', 'c

In [11]:
print('RootPCTokenizer')
rootPCTokenizer = RootPCTokenizer()
print('len(rootPCTokenizer.vocab): ', len(rootPCTokenizer.vocab))
stats['RootPCTokenizer'] = {
    'vocab_size': len(rootPCTokenizer.vocab),
    'seq_lens': []
}
toks_rpc = rootPCTokenizer(gjt_pieces)
print('example sentence length: ', len(toks_rpc['tokens'][0]))
print(toks_rpc['tokens'][0])
print(toks_rpc['ids'][0])
for t in toks_rpc['tokens']:
    stats['RootPCTokenizer']['seq_lens'].append( len(t) )
stats['RootPCTokenizer']['mean_len'] = np.mean(stats['RootPCTokenizer']['seq_lens'])
stats['RootPCTokenizer']['std_len'] = np.std(stats['RootPCTokenizer']['seq_lens'])
print('mean: ', stats['RootPCTokenizer']['mean_len'])
print('std: ', stats['RootPCTokenizer']['std_len'])

RootPCTokenizer
len(rootPCTokenizer.vocab):  46
example sentence length:  226
['bar', 'position_0x0', 'chord_root_7', 'chord_pc_10', 'chord_pc_2', 'chord_pc_5', 'bar', 'position_0x0', 'chord_root_0', 'chord_pc_4', 'chord_pc_7', 'chord_pc_10', 'bar', 'position_0x0', 'chord_root_5', 'chord_pc_9', 'chord_pc_0', 'chord_pc_4', 'bar', 'position_0x0', 'chord_root_5', 'chord_pc_9', 'chord_pc_0', 'chord_pc_4', 'bar', 'position_0x0', 'chord_root_7', 'chord_pc_10', 'chord_pc_2', 'chord_pc_5', 'bar', 'position_0x0', 'chord_root_0', 'chord_pc_4', 'chord_pc_7', 'chord_pc_10', 'bar', 'position_0x0', 'chord_root_5', 'chord_pc_9', 'chord_pc_0', 'chord_pc_4', 'bar', 'position_0x0', 'chord_root_5', 'chord_pc_9', 'chord_pc_0', 'chord_pc_4', 'bar', 'position_0x0', 'chord_root_4', 'chord_pc_7', 'chord_pc_11', 'chord_pc_2', 'bar', 'position_0x0', 'chord_root_9', 'chord_pc_1', 'chord_pc_4', 'chord_pc_7', 'bar', 'position_0x0', 'chord_root_2', 'chord_pc_5', 'chord_pc_9', 'chord_pc_0', 'bar', 'position_0x0', 'c

In [15]:
print('GCTRootPCTokenizer')
gctRootPCTokenizer = GCTRootPCTokenizer()
print('len(gctRootPCTokenizer.vocab): ', len(gctRootPCTokenizer.vocab))
stats['GCTRootPCTokenizer'] = {
    'vocab_size': len(gctRootPCTokenizer.vocab),
    'seq_lens': []
}
toks_gct_rpc = gctRootPCTokenizer(gjt_pieces)
print('example sentence length: ', len(toks_gct_rpc['tokens'][0]))
print(toks_gct_rpc['tokens'][0])
print(toks_gct_rpc['ids'][0])
for t in toks_gct_rpc['tokens']:
    stats['GCTRootPCTokenizer']['seq_lens'].append( len(t) )
stats['GCTRootPCTokenizer']['mean_len'] = np.mean(stats['GCTRootPCTokenizer']['seq_lens'])
stats['GCTRootPCTokenizer']['std_len'] = np.std(stats['GCTRootPCTokenizer']['seq_lens'])
print('mean: ', stats['GCTRootPCTokenizer']['mean_len'])
print('std: ', stats['GCTRootPCTokenizer']['std_len'])

GCTRootPCTokenizer
len(gctRootPCTokenizer.vocab):  46
example sentence length:  226
['bar', 'position_0x0', 'chord_root_10', 'chord_pc_2', 'chord_pc_5', 'chord_pc_7', 'bar', 'position_0x0', 'chord_root_0', 'chord_pc_4', 'chord_pc_7', 'chord_pc_10', 'bar', 'position_0x0', 'chord_root_9', 'chord_pc_0', 'chord_pc_4', 'chord_pc_5', 'bar', 'position_0x0', 'chord_root_9', 'chord_pc_0', 'chord_pc_4', 'chord_pc_5', 'bar', 'position_0x0', 'chord_root_10', 'chord_pc_2', 'chord_pc_5', 'chord_pc_7', 'bar', 'position_0x0', 'chord_root_0', 'chord_pc_4', 'chord_pc_7', 'chord_pc_10', 'bar', 'position_0x0', 'chord_root_9', 'chord_pc_0', 'chord_pc_4', 'chord_pc_5', 'bar', 'position_0x0', 'chord_root_9', 'chord_pc_0', 'chord_pc_4', 'chord_pc_5', 'bar', 'position_0x0', 'chord_root_7', 'chord_pc_11', 'chord_pc_2', 'chord_pc_4', 'bar', 'position_0x0', 'chord_root_9', 'chord_pc_1', 'chord_pc_4', 'chord_pc_7', 'bar', 'position_0x0', 'chord_root_5', 'chord_pc_9', 'chord_pc_0', 'chord_pc_2', 'bar', 'position_0x

In [13]:
print('GCTSymbolTokenizer')
gctSymbolTokenizer = GCTSymbolTokenizer()
print('training')
gctSymbolTokenizer.fit( gjt_pieces )
print('len(gctSymbolTokenizer.vocab): ', len(gctSymbolTokenizer.vocab))
stats['GCTSymbolTokenizer'] = {
    'vocab_size': len(gctSymbolTokenizer.vocab),
    'seq_lens': []
}
toks_gct_symb = gctSymbolTokenizer(gjt_pieces)
print('example sentence length: ', len(toks_gct_symb['tokens'][0]))
print(toks_gct_symb['tokens'][0])
print(toks_gct_symb['ids'][0])
for t in toks_gct_symb['tokens']:
    stats['GCTSymbolTokenizer']['seq_lens'].append( len(t) )
stats['GCTSymbolTokenizer']['mean_len'] = np.mean(stats['GCTSymbolTokenizer']['seq_lens'])
stats['GCTSymbolTokenizer']['std_len'] = np.std(stats['GCTSymbolTokenizer']['seq_lens'])
print('mean: ', stats['GCTSymbolTokenizer']['mean_len'])
print('std: ', stats['GCTSymbolTokenizer']['std_len'])

GCTSymbolTokenizer
training
len(gctSymbolTokenizer.vocab):  210
example sentence length:  110
['bar', 'position_0x0', '[10  0  4  7  9]', 'bar', 'position_0x0', '[ 0  0  4  7 10]', 'bar', 'position_0x0', '[9 0 3 7 8]', 'bar', 'position_0x0', '[9 0 3 7 8]', 'bar', 'position_0x0', '[10  0  4  7  9]', 'bar', 'position_0x0', '[ 0  0  4  7 10]', 'bar', 'position_0x0', '[9 0 3 7 8]', 'bar', 'position_0x0', '[9 0 3 7 8]', 'bar', 'position_0x0', '[7 0 4 7 9]', 'bar', 'position_0x0', '[ 9  0  4  7 10]', 'bar', 'position_0x0', '[5 0 4 7 9]', 'bar', 'position_0x0', '[ 7  0  4  7 10]', 'bar', 'position_0x0', '[4 0 3 7 8]', 'bar', 'position_0x0', '[5 0 4 7 9]', 'position_2x0', '[ 7  0  4  7 10]', 'bar', 'position_0x0', '[10  0  4  7  9]', 'position_0x0', '[ 0  0  4  7 10]', 'position_0x0', '[5 0 4 7]', 'bar', 'position_0x0', '[ 0  0  4  7 10]', 'bar', 'position_0x0', '[10  0  4  7  9]', 'bar', 'position_0x0', '[ 0  0  4  7 10]', 'bar', 'position_0x0', '[9 0 3 7 8]', 'bar', 'position_0x0', '[9 0 3 7

In [16]:
print('GCTRootTypeTokenizer')
gctRootTypeTokenizer = GCTRootTypeTokenizer()
print('training')
gctRootTypeTokenizer.fit( gjt_pieces )
print('len(gctRootTypeTokenizer.vocab): ', len(gctRootTypeTokenizer.vocab))
stats['GCTRootTypeTokenizer'] = {
    'vocab_size': len(gctRootTypeTokenizer.vocab),
    'seq_lens': []
}
toks_gct_rt = gctRootTypeTokenizer(gjt_pieces)
print('example sentence length: ', len(toks_gct_rt['tokens'][0]))
print(toks_gct_rt['tokens'][0])
print(toks_gct_rt['ids'][0])
for t in toks_gct_rt['tokens']:
    stats['GCTRootTypeTokenizer']['seq_lens'].append( len(t) )
stats['GCTRootTypeTokenizer']['mean_len'] = np.mean(stats['GCTRootTypeTokenizer']['seq_lens'])
stats['GCTRootTypeTokenizer']['std_len'] = np.std(stats['GCTRootTypeTokenizer']['seq_lens'])
print('mean: ', stats['GCTRootTypeTokenizer']['mean_len'])
print('std: ', stats['GCTRootTypeTokenizer']['std_len'])

GCTRootTypeTokenizer
training
len(gctRootTypeTokenizer.vocab):  71
example sentence length:  149
['bar', 'position_0x0', 'chord_root_10', '[0 4 7 9]', 'bar', 'position_0x0', 'chord_root_0', '[ 0  4  7 10]', 'bar', 'position_0x0', 'chord_root_9', '[0 3 7 8]', 'bar', 'position_0x0', 'chord_root_9', '[0 3 7 8]', 'bar', 'position_0x0', 'chord_root_10', '[0 4 7 9]', 'bar', 'position_0x0', 'chord_root_0', '[ 0  4  7 10]', 'bar', 'position_0x0', 'chord_root_9', '[0 3 7 8]', 'bar', 'position_0x0', 'chord_root_9', '[0 3 7 8]', 'bar', 'position_0x0', 'chord_root_7', '[0 4 7 9]', 'bar', 'position_0x0', 'chord_root_9', '[ 0  4  7 10]', 'bar', 'position_0x0', 'chord_root_5', '[0 4 7 9]', 'bar', 'position_0x0', 'chord_root_7', '[ 0  4  7 10]', 'bar', 'position_0x0', 'chord_root_4', '[0 3 7 8]', 'bar', 'position_0x0', 'chord_root_5', '[0 4 7 9]', 'position_2x0', 'chord_root_7', '[ 0  4  7 10]', 'bar', 'position_0x0', 'chord_root_10', '[0 4 7 9]', 'position_0x0', 'chord_root_0', '[ 0  4  7 10]', 'posi

In [26]:
# print stats
tokenizers = ['ChordSymbolTokenizer', 'GCTSymbolTokenizer',\
              'RootTypeTokenizer', 'GCTRootTypeTokenizer',\
              'PitchClassTokenizer', 'RootPCTokenizer', 'GCTRootPCTokenizer'
              ]
print('Tokenizer             \t', 'vocab size \t', 'mean_len\t', 'std_len')
for tok in tokenizers:
    m = stats[tok]['mean_len']
    s = stats[tok]['std_len']
    print( tok + ': \t', '\t'+str( stats[tok]['vocab_size'] )+'\t', f'{m:.2f}\t\t', f'{s:.2f}')

Tokenizer             	 vocab size 	 mean_len	 std_len
ChordSymbolTokenizer: 	 	370	 120.49		 43.32
GCTSymbolTokenizer: 	 	210	 120.49		 43.32
RootTypeTokenizer: 	 	63	 164.89		 60.34
GCTRootTypeTokenizer: 	 	71	 164.89		 60.34
PitchClassTokenizer: 	 	34	 255.11		 95.20
RootPCTokenizer: 	 	46	 255.11		 95.20
GCTRootPCTokenizer: 	 	46	 255.11		 95.20
